# PyArrow

- https://arrow.apache.org/
- Pasos de instalación
- Descripción del producto, modo de funcionamiento, mostrar cómo describir los esquemas
- Mostrar cómo importar los datos de Stackoverflow
- Mostrar posibles optimizaciones para realizar RQ1 a RQ4 vistos en la sesión 2


## Pasos de Instalación

La instalación de PyArrow (a 26 de noviembre de 2024 se recomienda la versión `18.1`) es simple, y viene detallada en su [página web](https://arrow.apache.org/install/), estando disponible (a la misma fecha) para Python desde su versión `3.9` hasta la `3.13`.

Existen tres paquetes de PyArrow disponibles para Python, con menor o mayor cantidad de componentes de Arrow integrados, que podemos instalar, aunque también podemos instalar estos componentes individualmente. Estos paquetes son los siguientes:

**Componente** | **Paquete** | `pyarrow-core` | `pyarrow` | `pyarrow-all`
---------|----------|:--------:|:-------:|:-------:
Core | pyarrow-core | ✅ | ✅ | ✅
Parquet | libparquet |  | ✅ | ✅
Dataset | libarrow-dataset |  | ✅ | ✅
Acero | libarrow-acero |  | ✅ | ✅
Substrait | libarrow-substrait |  | ✅ | ✅
Flight | libarrow-flight |  |  | ✅
Flight SQL | libarrow-flight-SQL |  |  | ✅
Gandiva | libarrow-gandiva |  |  | ✅

Según el entorno de desarrollo y el entorno virtual que utilicemos usaremos **pip** o **conda** para instalar el paquete:

- Instalación con **pip**: `pip install pyarrow==18.1.*`
- Instalación con **conda**: `conda install pyarrow=18.1.* -c conda-forge`

En la instalación se incluyen los binarios de las librerías `Apache Arrow` y `Apache Parquet` de C++. 

PyArrow tiene las siguientes dependencias opcionales:
- `NumPyy 1.16.6` o mayor
- `pandas 1.0` o mayor
- `cffi` (permite interactuar con código de C en Python)

Además también podemos acceder al código fuente de Arrow y sus diferentes implementaciones en su [repositorio de Github](https://github.com/apache/arrow).

### Instalación de Librerías Necesarias 
A continuación, instalamos las librerías necesarias para ejecutar el notebook.

In [2]:
%pip install numpy
%pip install pandas
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## ¿Qué es PyArrow?
PyArrow es la implementación de **Apache Arrow** para **Python**, por lo que para poder explicar qué es PyArrow debemos conocer primero Apache Arrow.

Apache Arrow es, según su página web, una plataforma de desarrollo de software para construir aplicaciones de gran rendimiento que procesan y transportan grandes conjuntos de datos. Está diseñado para mejorar el rendimiento de los algoritmos de análisis y para eficiencia al mover datos de un sistema o lenguaje de programación a otro. 

En la práctica, se utiliza para mejorar el rendimiento y la interoperabilidad entre diferentes herramientas de procesamiento de datos dado que el formato Arrow tiene implementación en la mayoría de lenguajes de programación modernos y más utilizados, como Python, R, Julia, Rust, Go, Java, C++, C, C# o Ruby.

Arrow destaca por la forma en la que almacena los datos en memoría, ya que lo hace en formato columnar, lo que permite un acceso más rápido y eficiente en grandes cargas de trabajo analíticas, permitiendo representar conjuntos de datos estructurados en formatos semejantes a tablas. Además, es *language-agnostic*, por lo que soporta diferentes lenguajes y facilita el intercambio de datos entre diferentes aplicaciones sin la necesidad de una gran cantidad de recursos. También aseguran la retrocompatibilidad del formato al añadir solo actualizaciones no destructivas, como la adición de nuevos tipos de datos.

PyArrow es la implementación de Apache Arrow en Python. Es una biblioteca que permite trabajar con el formato Arrow y su ecosistema desde Python, y está construida sobre la librería de Arrow en C++ (similar a lo que ocurre con la librería `numpy`), permitiendo así un tratamiento de datos más rápido.

PyArrow permite la creación y manipulación de estructuras de datos Arrow directamente en Python, permitiendo trabajar con ellas con operaciones paralelas y destacando su fácil conversión en `DataFrames` de la librería `pandas` y con `pySpark`, por lo que permite enlazar con las técnicas de análisis de datos distribuidos que ofrece Spark. De igual forma, PyArrow permite trabajar con los formatos de datos más comunes, como son CSV y Parquet.

### Funcionamiento
Como hemos dicho, PyArrow es un paquete de Python que funciona como una interfaz para manejar datos usando el formato Apache Arrow. Al funcionar sobre la librería de Apache Arrow para C++, es rápido y eficiente a la hora de manejar datos tanto en la memoría como en el disco, al aprovechar las ventajas tanto de Apache Arrow como de C++.

#### Estructuras Básicas
Las dos estructuras básicas para manejar datos de PyArrow son las `Table`s y los `Array`s. Una `Table` representa datos en forma de tabla, de forma similar a los `DataFrames` de pandas, mientras `Array` representaría una única columna de datos con un único tipo, de forma paralela a las `Series` de pandas. Sin embargo, existe una diferencia principal entre ellas, y es que las estructuras de PyArrow son **inmutables**, lo cual es clave a la hora de garantizar la integridad de los datos y mejorar el rendimiento.

Podemos crear una `Table` a partir de  un diccionario de Python, de forma muy parecida a como lo hacíamos en pandas, y un `Array` a partir de una lista:

In [3]:
import numpy as np
import pyarrow as pa

array = pa.array([1, 2, 3, 4, 5])

ex_dict = {
    "manzanas": [2,7,3,4],
    "kiwis": [3,2,1,5],
    "limones": [2,3,8,9],
    "brevas": [6,3,1,9],
}

table = pa.Table.from_pydict(ex_dict)
table

pyarrow.Table
manzanas: int64
kiwis: int64
limones: int64
brevas: int64
----
manzanas: [[2,7,3,4]]
kiwis: [[3,2,1,5]]
limones: [[2,3,8,9]]
brevas: [[6,3,1,9]]

También podemos generar `Table`s mediante la combinación de diferentes `Array`s:

In [4]:
np.random.seed(42069)
arrays = [list(map(round, np.random.normal(10,3,10))) for _ in range(4)]
table = pa.Table.from_arrays(arrays, names = ['manzanas', 'kiwis', 'limones', 'brevas'])
table

pyarrow.Table
manzanas: int64
kiwis: int64
limones: int64
brevas: int64
----
manzanas: [[10,9,11,12,13,8,12,9,4,14]]
kiwis: [[4,10,11,5,8,14,11,9,5,12]]
limones: [[9,12,13,10,6,16,6,9,11,9]]
brevas: [[12,8,15,15,14,10,9,9,9,6]]

Como vemos, la presentación de las tablas de PyArrow no es tan clara y estelística como podríamos desear. Podemos solucionar esto mediante una de las grandes ventajas de PyArrow, que es su compatibilidad con pandas:

In [9]:
import pandas as pd

df = table.to_pandas()
df


,manzanas,kiwis,limones,brevas
0,10,4,9,12
1,9,10,12,8
2,11,11,13,15
3,12,5,10,15
4,13,8,6,14
5,8,14,16,10
6,12,11,6,9
7,9,9,9,9
8,4,5,11,9
9,14,12,9,6


Esta compatibilidad es bidireccional, es decir, también podemos una `Table` de PyArrow desde un `DataFrame` de pandas:

In [11]:
table2 = pa.Table.from_pandas(df)
table2

pyarrow.Table
manzanas: int64
kiwis: int64
limones: int64
brevas: int64
----
manzanas: [[10,9,11,12,13,8,12,9,4,14]]
kiwis: [[4,10,11,5,8,14,11,9,5,12]]
limones: [[9,12,13,10,6,16,6,9,11,9]]
brevas: [[12,8,15,15,14,10,9,9,9,6]]

- CSV
    - Batchs
- Particionado
- Formato propio

### Descripción de Esquemas

## Importar los Datos de Stackoverflow

## Ejercicios
### RQ1: Distribution of Questioners
We plot the histogram of questioners in Figure 3. The graph shows the number of developers that ask a given num- ber of questions, and its y-axis is in log-scale. From the graph, we notice that most developers (33,907 out of 44,087 developers) only ask one question. Only about 23.1% of the developers ask two or more questions. The number of developers that ask questions reduces exponentially as we consider a higher number of posted questions. Only 1.6% of the developers ask more than 5 questions. 

The result shows that there are few “regular” questioners on StackOverflow. This is possibly because many questions have already been asked before and users could find answers to them by just looking into the various pages on StackOverflow or other question and answer sites via search engines

### RQ2: Distribution of Answerers
We plot the histogram of answerers in Figure 3. The graph shows the number of developers that answer a given num- ber of questions and its y-axis is also in log-scale. From the graph, we notice that most developers (28,578 out of 44,087 developers) only answer one question. About one thousand developers (2.3%) do not answer any questions. Only about 35.2% of the developers answer two or more questions. The number of developers that answer questions reduces expo- nentially as we consider a higher number of answers. On- ly 7.8% of the developers answer more than 5 questions. The highest number of questions a developer answers in our dataset is 178. There is only one developer that answers this many questions.

Compared with the distribution of questioners, the distribution of answers is different. The number of developers that answer a substantial number of questions (> 5) is more than the number of developers that ask a substantial number of questions (> 5)—3,424 (7.8%) versus 701 (1.6%). This may imply that many developers on StackOverflow are interested

### RQ3: Segregation of StackOverflow Com munity
To answer this research question, we investigate the proportion of posts that various developers make that are answers to some questions. We show this in Figure 5. We notice that a majority of developers only ask questions but do not answers them (83.2%, 36,672 developers). Thus, we could divide the StackOverflow community into two groups: people that only ask questions, and those that answer one or more questions. The first group is the majority.

We also note another peak in Figure 5: These are developers (2,956 of 44,087 developers), with 50-59% of posts being answers. These correspond to ideal developers that contribute answers to the community as much as requesting answers from the community.

### RQ4: RQ4: Reciprocity in StackOverflow
To answer the fourth research question, we investigate the help graph. A help graph is a directed graph, where each developer is a node, and the node corresponding to a developer D1 is linked to that of D2 if D1 answers a question posted by D2. We would like to investigate how often two developers D1 and D2 are connected by two edges, one from D1 to D2 and the other from D2 to D1. We find that there are only a few of such developers (23 pairs). We highlight a few in Table 1. The table contains the identifiers of the helpers and helpees that reciprocate. From the result, we hypothesize that developers tend to help anyone no matter if they have helped him or her before and StackOverflow tends to benefit the community as a whole.